In [3]:
!pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
!pip install datasets
from datasets import load_metric
!pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [4]:
# from huggingface_hub import notebook_login
# notebook_login()

In [6]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



1776


In [7]:
# from datasets import Dataset, load_metric

In [8]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [9]:
len(train)

17

In [10]:
train['label'].value_counts()

0    6
2    6
1    5
Name: label, dtype: int64

In [11]:
test['label'].value_counts()

0    597
2    594
1    585
Name: label, dtype: int64

In [14]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a9e54058642b564390eac3e75602160b67e89b3560d563e575562b854dd1a4cf
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [20]:
train['embeddings'] = train['text'].apply(model.encode)
test['embeddings'] = test['text'].apply(model.encode)

In [21]:
test

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder,embeddings
1690,0f0c7e4ba272bfca9c1c535050f609e1bd0cfd8c,cbeust$testng-eclipse,Updated property page to match functionality o...,2,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,"[0.020912003, 0.1329397, -0.1853192, -0.059800..."
87,46349a89d9d332e1f7c9f976f65efc79ea46cd29,hbase,HBASE-6667 TestCatalogJanitor occasionally fai...,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.12775995, 0.218312, 0.048813872, 0.0453074..."
518,d47f33629bcc15db2d0f9d75289b0d0da4850799,drools,add support for maven version ranges--,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.18049385, -0.0006577305, -0.16153003, -0.0..."
202,cb51d3c5762e6f2808f9ab358943b3dc85e9bd95,elasticsearch,Sorting on _score in the URI format is reversed,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.04494988, 0.20357062, -0.14293493, 0.134963..."
890,e3d1a1dda22723fc896bfc96c6db57c500faf208,spring-framework,@Resource injection points support @Lazy as we...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,"[-0.060315046, -0.26188236, -0.036801048, 0.05..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,a29e41b9fa93162b7731ed2d45a2ac368384decd,spring-framework,Fix Jackson @JSONView when using XML- serializ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.06882839, 0.26821515, -0.013796627, -0.004..."
1299,b0caefee80a42d6737b3a255316487b668fab104,Delta Spike,DELTASPIKE-278 add category to Message API\n,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.0046929754, 0.2061885, -0.0643009, 0.00398..."
1768,407ba7ef39b8a60e6153b64bb18ba54fa723937b,isa-tools$isacreator,More additions to merge the new ontology brows...,2,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,"[0.06375633, 0.14423685, -0.19336137, -0.00494..."
1468,8fbc82a7314ed46df0f9e2371f065ddfc0282c9a,Vala,gdk-2.0: add many missing type_arguments\n\nPa...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.07194574, 0.15438816, -0.00027219296, 0.092..."


In [22]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [23]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [25]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.43355855855855857
Logistic Regression Precision: 0.4405982143979174
Logistic Regression Recall: 0.43355855855855857
Logistic Regression Recall: 0.42582566567480706
